In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import random
import tensorflow as tf
from model_structures import *
from data_loader import *
from loss_func import *
import datetime
import scipy.io as sio

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]='1'

In [ ]:
opt = {}

'''cosmos'''
opt['train_path'] = '/home/sylar/data/invivo/data'
opt['train_img1_path'] = '/msk_arr'
opt['train_img2_path'] = '/phs_unwrap'
opt['train_label_path'] = '/phs_tissue'

opt['patch_size'] = (160,160,160)
opt['reso'] = (1.06,1.06,1.06)

opt['lbd0'] = 100   
opt['lbd1'] = 0.1
opt['thr'] = 0.3
opt['c_iter'] = 5
opt['rad'] = [12,10,8,6,4,2]

opt['unrely_tol'] = 1e-3 
opt['iter'] = 5
opt['batch_size'] = 1
opt['channels'] = len(opt['rad']) + 1
opt['img_shape'] = opt['patch_size'] + (len(opt['rad'])+1,)
opt['in_shape'] = opt['patch_size'] + (1,)
opt['rnd_crop'] = False

opt['model_restored_epoch'] = 77 

opt['loss'] = nrmse

opt['display_nums'] = [80,50,30]

opt['model_save_path'] = '/home/sylar/data/results/model1_revision/br_c8_s10_smv53_cg_trun_grad'
opt['out_dir'] = '/home/sylar/data/invivo/data/at_vs_test/'
opt['checkpoint_path'] = opt['model_save_path']+"/cp{epoch}"

In [ ]:
opt['train_index'] = list(range(1,25))

In [ ]:
opt['test_data'] = Data_loaders_invivo1(opt)
x1_test, x2_test, y_test = opt['test_data'].next([0],opt)
print('testing data size: '+str(opt['test_data'].data_size))

In [ ]:
'''loading data preview'''
ch,cw,cd = findcenter3d(y_test[0])
aa=cw
dd=cd

f, axarr = plt.subplots(2, 3, figsize=(20, 10))
axarr[1,0].imshow(np.flip(np.transpose(x1_test[0,:,aa,:,0].squeeze()),0))
axarr[1,0].axis('off')
axarr[0,2].imshow(np.transpose(y_test[0,:,:,dd].squeeze()))
axarr[0,2].axis('off')
axarr[0,0].imshow(np.transpose(x1_test[0,:,:,dd,0].squeeze()))
axarr[0,0].axis('off')
axarr[1,1].imshow(np.flip(np.transpose(x2_test[0,:,aa,:].squeeze()),0))
axarr[1,1].axis('off')
axarr[1,2].imshow(np.flip(np.transpose(y_test[0,:,aa,:].squeeze()),0))
axarr[1,2].axis('off')
axarr[0,1].imshow(np.transpose(x2_test[0,:,:,dd].squeeze()))
axarr[0,1].axis('off')
plt.show()

In [ ]:
# tf.compat.v1.disable_eager_execution()
model = {}

model['vnet'] = unet_at1(opt, 1)
x = Input(shape=opt['img_shape'])

model['vnet'].load_weights(opt['checkpoint_path'].format(epoch = opt['model_restored_epoch']))


In [ ]:
nr_im = opt['test_data'].data_size
start_time = datetime.datetime.now()

smv,trun = smv_iter_array(opt)

nr_im_per_epoch = int(np.ceil(nr_im/ opt['batch_size']) * opt['batch_size'])
if nr_im < opt['batch_size']:
    order = list(range(nr_im)) + [nr_im-1]*(opt['batch_size']-nr_im)
else:
    order = list(range(nr_im)) + list(range(nr_im_per_epoch - nr_im))

avg_img_cost = []
for block_i in range(0, opt['test_data'].data_size, opt['batch_size']):
    
    indices = order[block_i:block_i + opt['batch_size']]
    x1_test, x2_test, y_test = opt['test_data'].next(indices,opt)
    x = np.concatenate((x1_test, x2_test),axis=-1)

    print('-----subject-----'+str(block_i+1)+'/'+str(nr_im_per_epoch))
    L = tf.zeros_like(x[...,1:2])
    
    msk = L
    for mm in range(len(opt['rad'])):
        msk = msk + x[...,mm:mm+1]
        
    L = conjgrad_br_trun(x[...,0:len(opt['rad'])], L,x[...,len(opt['rad']):len(opt['rad'])+1],L,smv,trun, opt)
    for it in range(opt['iter']):
        net_out = model['vnet'].predict(L)
        L = conjgrad_br_trun(x[...,0:len(opt['rad'])],L,x[...,len(opt['rad']):len(opt['rad'])+1],net_out,smv,trun,opt)

    m_loss = np.linalg.norm(L*msk-y_test)/np.linalg.norm(y_test)
    # Plot the progress
    print ("[Batch %d/%d] [Model loss: %f]" % (block_i+1, nr_im_per_epoch, m_loss))
    avg_img_cost.append(m_loss)

    s = 10
    save_nii(msk.numpy().squeeze(), opt['reso'],  opt['out_dir'], 'pocsnet_msk')
    save_nii((net_out*msk.numpy()/s).squeeze(), opt['reso'],  opt['out_dir'], 'pocsnet_net_phs')
    save_nii((L.numpy()*msk.numpy()/s).squeeze(), opt['reso'],  opt['out_dir'], 'pocsnet_model_phs')
    
    display_slice(opt['display_nums'], net_out/s, L.numpy()/s, y_test/s)
epoch_loss = np.mean(avg_img_cost)

print("Testing_cost=", "{:.5f}".format(epoch_loss))
            
elapsed_time = datetime.datetime.now() - start_time
print(elapsed_time)

In [ ]:
np.std(avg_img_cost)